In [1]:
# repeat ICA for 100 times
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from scipy.sparse import csr_matrix
from matplotlib.lines import Line2D
from sklearn import cluster, metrics
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from itertools import combinations_with_replacement
from sklearn.decomposition import FastICA

In [2]:
# Data was filtered to preserve one sample type per experimental condition 
data = pd.read_csv("filtered_data.csv",index_col=0,nrows=1000)

In [3]:
# Non-standard normalization in order to preserve sparsity, as z-scores fail to execute
# due to mathematical error of dividing by zero
scaler = StandardScaler(with_mean=False)
scaler.fit(data.transpose())
norm_data = scaler.transform(data.transpose())

In [6]:
A_marix_collection={}
M_marix_collection={}


n=0
while n<100:
    ica_transformer = FastICA(n_components=30,random_state=0).fit(norm_data[0:300].transpose())
    M = pd.DataFrame(ica_transformer.transform(norm_data[0:300].transpose()))
    A = pd.DataFrame(ica_transformer.mixing_)
    
  
    A_marix_collection[n]=A
    M_marix_collection[n]=M
    
    
    n=n+1

In [ ]:
n=0

M1=M_marix_collection[0]
A1=A_marix_collection[0]

while n<99:
    M1=M
    M2=M_marix_collection[n+1]
    A1=A
    A2=A_marix_collection[n+1]

    metrics = pd.DataFrame(index = range(0,len(M1.columns)),columns= range(0,len(M1.columns)))
    for i in range(0,len(M1.columns)):
        for j in range(0,len(M2.columns)):
            metrics.loc[i][j] = abs(pearsonr(M1[i],M2[j])[0])
    metrics= metrics.fillna(0)

    
    
    M = pd.DataFrame(index=M1.index)
    
    for i,item in metrics.iteritems():
        for j in item.index:
            if item[j] == max(item) and abs(max(item)) > .9:
                M1_abs_max = float(M1[i].loc[M1[i].abs().nlargest(1).index])
                M2_abs_max = float(M2[j].loc[M2[j].abs().nlargest(1).index])
                A1_abs_max = float(A1[i].loc[A1[i].abs().nlargest(1).index])
                A2_abs_max = float(A2[j].loc[A2[j].abs().nlargest(1).index])
                if M1_abs_max < 0:
                    M1_corrected = -M1[i]
                else:
                    M1_corrected = M1[i]
                if M2_abs_max < 0:
                    M2_corrected = -M2[i]
                else:
                    M2_corrected = M2[i]
                if A1_abs_max < 0:
                    A1_corrected = -A1[i]
                else:
                    A1_corrected = A1[i]
                if A2_abs_max < 0:
                    A2_corrected = -A2[i]
                else:
                    A2_corrected = A2[i]
                M[str(i)+"_"+str(j)] = (M1_corrected+M2_corrected)/2 
                A[str(i)+"_"+str(j)] = (A1_corrected+M2_corrected)/2 
    
    
    
    
n=n+1